In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
import os
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from keras.utils import to_categorical
import pickle

In [15]:
SIZE = 128

def load_images_and_labels(directory):
    images = []
    labels = []

    for label in os.listdir(directory):
        label_path = os.path.join(directory, label)
        if os.path.isdir(label_path):
            for img_path in glob.glob(os.path.join(label_path, "*.png")):
                img = cv2.imread(img_path, cv2.IMREAD_COLOR)
                img = cv2.resize(img, (SIZE, SIZE))
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                images.append(img)
                labels.append(label)

    return np.array(images), np.array(labels)

# Path to the training and testing data directories
train_data_directory = "/kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/train"
test_data_directory = "/kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/test"

# Load training and testing data
train_images, train_labels = load_images_and_labels(train_data_directory)
test_images, test_labels = load_images_and_labels(test_data_directory)

In [16]:

# Save the preprocessed training images to a NumPy array
train_images_filename = "/kaggle/working/train_images.npy"
np.save(train_images_filename, train_images)

# Save the training labels to a CSV file
train_labels_filename = "/kaggle/working/train_labels.csv"
train_labels_df = pd.DataFrame({"label": train_labels})
train_labels_df.to_csv(train_labels_filename, index=False)

# Save the preprocessed test images to a NumPy array
test_images_filename = "/kaggle/working/test_images.npy"
np.save(test_images_filename, test_images)

# Save the test labels to a CSV file
test_labels_filename = "/kaggle/working/test_labels.csv"
test_labels_df = pd.DataFrame({"label": test_labels})
test_labels_df.to_csv(test_labels_filename, index=False)

In [17]:
# Load the preprocessed images and labels from the Kaggle dataset
train_images = np.load("/kaggle/working/train_images.npy")
train_labels = pd.read_csv("/kaggle/working/train_labels.csv")["label"].values

test_images = np.load("/kaggle/working/test_images.npy")
test_labels = pd.read_csv("/kaggle/working/test_labels.csv")["label"].values

In [20]:
from sklearn.preprocessing import LabelEncoder
# Initialize the LabelEncoder
labelEn = LabelEncoder()

# Fit and transform the training labels
train_labels_encoded = labelEn.fit_transform(train_labels)

# Transform the test labels using the same encoder
test_labels_encoded = labelEn.transform(test_labels)

# Create a dictionary mapping classes to their encoded values
class_mapping = dict(zip(labelEn.classes_, range(len(labelEn.classes_))))

# Print the mapping
print(class_mapping)

{'freshapples': 0, 'freshbanana': 1, 'freshoranges': 2, 'rottenapples': 3, 'rottenbanana': 4, 'rottenoranges': 5}


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Reshape the images
x_train_flattened = x_train.reshape(x_train.shape[0], -1)
x_test_flattened = x_test.reshape(x_test.shape[0], -1)

# Create and train the logistic regression model
logistic_model = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
logistic_model.fit(x_train_flattened, train_labels_encoded)

# Make predictions on the test data
test_predictions = logistic_model.predict(x_test_flattened)

# Calculate the test accuracy
test_accuracy = accuracy_score(test_labels_encoded, test_predictions)
print("Logistic Regression Test Accuracy:", test_accuracy)

In [ ]:
# check for a selected image
n = 356
img = x_test[n]
plt.imshow(img)
plt.show()

# Expand dims for the input image
input_img = img[np.newaxis, ...]

# Reshape the input image features
input_img_features = input_img.reshape(1, -1)

# Make a prediction using the logistic regression model
prediction_logistic = logistic_model.predict(input_img_features)[0]

# Reverse the label encoder to the original name
prediction_logistic = labelEn.inverse_transform([prediction_logistic])

# Print the prediction and the actual label for the image
print("The prediction for this image is:", prediction_logistic)
print("The actual label for this image is:", test_labels[n])

In [ ]:
# Save the logistic regression model to disk
filename = 'finalized_logistic_regression_model.pkl'
with open(filename, 'wb') as file:
    pickle.dump(logistic_model, file)

# Load the logistic regression model from disk
loaded_model = pickle.load(open(filename, 'rb'))

# Make predictions using the loaded model (for example)
loaded_prediction = loaded_model.predict(input_img_features)
print("Prediction using loaded model:", labelEn.inverse_transform([loaded_prediction]))